# TG4 - Ripple Ecosystem Backtest
## LSTM + Momentum Strategy for XRP/RLUSD Accumulation

In [ ]:
import sys
sys.path.insert(0, '../src')

from data_fetcher import DataFetcher
from backtester import Backtester
from strategies.ripple_momentum_lstm import generate_ripple_signals
from models.lstm_predictor import LSTMPredictor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Fetch Data from Exchanges

In [ ]:
fetcher = DataFetcher()

symbols = ['XRP/USDT', 'BTC/USDT']
data = {}

for sym in symbols:
    print(f"Fetching {sym} from Kraken...")
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 2000)
    if not df.empty:
        data[sym] = df
        print(f"  Got {len(df)} candles")
    else:
        print(f"  Failed to fetch {sym}")

## 2. Train LSTM Model

In [ ]:
if 'XRP/USDT' in data:
    close = data['XRP/USDT']['close'].values
    
    predictor = LSTMPredictor()
    train_end = int(len(close) * 0.8)
    
    print(f"Training on {train_end} samples...")
    predictor.train(close[:train_end], epochs=50)

## 3. Run Backtest

In [ ]:
if 'XRP/USDT' in data:
    signals = generate_ripple_signals(data, 'XRP/USDT')
    
    bt = Backtester(data)
    pf = bt.run_with_lstm_signals('XRP/USDT', signals)
    
    print(f"\n=== Backtest Results ===")
    print(f"Total Return: {pf.total_return():.2%}")
    print(f"Sharpe Ratio: {pf.sharpe_ratio():.2f}")
    print(f"Max Drawdown: {pf.max_drawdown():.2%}")
    print(f"Total Trades: {pf.trades.count()}")

## 4. Plot Equity Curve and Trades

In [ ]:
if 'XRP/USDT' in data:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Equity curve
    pf.value().plot(ax=axes[0], title='Portfolio Value Over Time')
    axes[0].set_ylabel('USD')
    
    # Price with signals
    close_series = data['XRP/USDT']['close']
    close_series.plot(ax=axes[1], label='XRP/USDT Price')
    
    buy_signals = signals[signals == True]
    axes[1].scatter(buy_signals.index, close_series.loc[buy_signals.index], 
                   marker='^', color='green', s=50, label='Buy Signal')
    
    axes[1].set_title('XRP/USDT with LSTM Signals')
    axes[1].set_ylabel('Price')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 5. Compare RSI-Only vs LSTM Strategy

In [ ]:
if 'XRP/USDT' in data:
    bt = Backtester(data)
    
    # RSI-only strategy
    pf_rsi = bt.run_momentum_strategy('XRP/USDT')
    
    print("=== RSI-Only Strategy ===")
    print(f"Total Return: {pf_rsi.total_return():.2%}")
    print(f"Sharpe Ratio: {pf_rsi.sharpe_ratio():.2f}")
    
    print("\n=== LSTM + Volume Strategy ===")
    print(f"Total Return: {pf.total_return():.2%}")
    print(f"Sharpe Ratio: {pf.sharpe_ratio():.2f}")